Genetic Jupyter Notebook

In [49]:
# Base imports
import random
import math

from collections import defaultdict
from typing import List

In [93]:
'''
Primary tool for running and presenting the results.

Tool for collecting population information for rendering purposes.
'''
class GeneticTool:

    def __init__(self, size: int=8, populationSize=32):
        self.gq = GeneticController()

        self.targetValue = random.uniform(-5000, 5000)

        # Creating our population
        self.population = []
        for value in range(populationSize):
            self.population.append(GeneticStructure())

    def run(self):
        iterationCount = 0
        while iterationCount < 10000:
            self.iteratePopulation()
            iterationCount += 1

    def iteratePopulation(self):
        newPopulation = []
        for idx in range(len(self.population)):
            pop = self.population[idx]
            value = self.gq.calculateOperandValue(pop)
            logarithmicTargetDistance = math.log10(abs(self.targetValue - value))

            # Breed with random chance
            breedChance = 0.05
            breedChance += logarithmicTargetDistance / 5

            if random.uniform(0, 1) < breedChance:
                breedTarget = random.choice(self.population)
                newPopulation.append(self.gq.breed(pop, breedTarget))


            # Randomly prune based on iteration age and distance from the target value
            deathChance = 0.4
            deathChance += logarithmicTargetDistance / 20

            if random.uniform(0, 1) > deathChance and len(self.population) < 1024:
                newPopulation.append(pop)

        self.population = newPopulation

    def getPopulationDistribution(self):
        dist = defaultdict(lambda: 0)
        for pop in self.population:
            for code in pop.geneticCode:
                dist[code] += 1

        return dist

In [91]:
'''
Structure for our genetic controller which will be used as essentially a singleton
'''

class GeneticController:

    def __init__(self, size: int=8):
        # Initializing our controller with a default size of 8.
        # Creating our operand function
        self.size = size
        self.geneticOperand = []

        for i in range(size):
            value = 0
            while value == 0:
                value = random.uniform(-10, 10)
            self.geneticOperand.append(value)

    def calculateOperandValue(self, gc: "GeneticStructure") -> int:
        output, idx = 0, 0
        for operand in self.geneticOperand:
            try:
                geneticCodeValue = gc.geneticCode[idx]
                if geneticCodeValue == '+':
                    output += operand
                elif geneticCodeValue == '-':
                    output -= operand
                elif geneticCodeValue == '*':
                    output *= operand
                elif geneticCodeValue == '/':
                    output /= operand
                elif geneticCodeValue == 'pow':
                    output = pow(output, operand)
                elif geneticCodeValue == 'sqrt':
                    output = pow(output, 1/operand)
            except:
                pass

            idx += 1

        if type(output) == complex:
            output = output.real

        if abs(output) > 100000000:
            return 0
        return round(output)

    def breed(self, gc1: "GeneticStructure", gc2: "GeneticStructure") -> GeneticStructure:
        newGeneticCode = []
        for idx in range(0, len(gc1.geneticCode)):
            newGeneticCode.append(random.choice([gc1.geneticCode[idx], gc2.geneticCode[idx]]))
        return GeneticStructure(size=gc1.size, geneticCode=newGeneticCode)

In [68]:
'''
Structure for genetic class
'''
class GeneticStructure:

    def __init__(self, size: int=8, geneticCode: List=[]):
        # Initializing our genetic structure with a default size of 8.
        # Size is essentially the number of variables within our genetic code.
        self.size = size
        self.geneticCode = []

        for i in range(self.size):
            
            value = random.choice(['+','-','*','/','pow','sqrt'])
            self.geneticCode.append(value)

In [94]:
geneticTool = GeneticTool()
print(geneticTool.getPopulationDistribution())
geneticTool.run()
print(geneticTool.getPopulationDistribution())

defaultdict(<function GeneticTool.getPopulationDistribution.<locals>.<lambda> at 0x000002D7E7B1BE50>, {'pow': 31, '/': 47, 'sqrt': 44, '-': 47, '*': 43, '+': 44})
defaultdict(<function GeneticTool.getPopulationDistribution.<locals>.<lambda> at 0x000002D7E7BCC0D0>, {'*': 1489, 'sqrt': 1512, '-': 1529, '+': 1514, 'pow': 1475, '/': 1457})
